# Count number of duplicated pairs in our dataset

In [16]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

import sqlite3
import argparse
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import os
from numpy.linalg import norm
import sys
from itertools import combinations
import random
import my_utils
import csv
import sys

from sklearn.model_selection import train_test_split

In [16]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

# Specify the path to your CSV file
csv_file_path = './datasets/dup_br_detection_data/dup_train.csv'

# Open the CSV file and create a CSV reader
labels = []
with open(csv_file_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # Iterate through each row in the CSV file
    # for row in csv_reader:
    #     print(row)
    line = 0
    
    for row in csv_reader:
        labels += [row[4]]

In [19]:
labels.count("yes")

89027

In [20]:
labels.count("No")

33270

In [3]:
line

122298

In [24]:
# Specify the path to your CSV file
csv_file_path = './datasets/dup_br_detection_data/dup_test.csv'

# Open the CSV file and create a CSV reader
labels = []
with open(csv_file_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # Iterate through each row in the CSV file
    # for row in csv_reader:
    #     print(row)
    line = 0
    
    for row in csv_reader:
        labels += [row[4]]
        line += 1

In [25]:
line

15289

In [26]:
# Specify the path to your CSV file
csv_file_path = './datasets/dup_br_detection_data/dup_valid.csv'

# Open the CSV file and create a CSV reader
labels = []
with open(csv_file_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # Iterate through each row in the CSV file
    # for row in csv_reader:
    #     print(row)
    line = 0
    
    for row in csv_reader:
        labels += [row[4]]
        line += 1

In [27]:
line

15288

# Count number of dups

In [3]:
database_path = "./dbrd.db"


conn = sqlite3.connect(database_path)
cursor = conn.cursor()


column_names = my_utils.get_column_names(conn, "spark")


In [3]:
num_dups = 0

for table in my_utils.table_names:
    union_find = my_utils.UnionFind()
    union_find.process_project(conn, table, column_names)
    
    roots = union_find.get_roots()
    print("Num roots", len(roots))
    num_dups += union_find.num_dups()
    
    

Processing spark


100%|██████████| 9579/9579 [00:00<00:00, 1806368.04it/s]


Num roots 290


100%|██████████| 290/290 [00:00<00:00, 28041.96it/s]


spark has 513.0 duplicated_pairs
Processing eclipse


...

spark_1day has 513.0 duplicated_pairs
Processing vscode


100%|██████████| 62092/62092 [00:00<00:00, 1634754.40it/s]


Num roots 2342


100%|██████████| 2342/2342 [00:00<00:00, 3372.48it/s]

vscode has 13931.0 duplicated_pairs


In [ ]:
union_find.processed

False

In [4]:
num_dups

202489.0

### Convert Hadoop data to csv

In [4]:
project_name = "hadoop"
union_find = my_utils.UnionFind()
union_find.process_project(conn, project_name, column_names)

Processing hadoop


100%|██████████| 14016/14016 [00:00<00:00, 1756314.68it/s]


In [5]:
duplicated_pairs = union_find.get_duplicated_pairs()

In [6]:
non_duplicated_pairs = my_utils.get_non_duplicated_pairs(union_find, conn, len(duplicated_pairs))

In [7]:
dataset = []

for bug_id1, bug_id2 in tqdm(duplicated_pairs+non_duplicated_pairs):
    text1 = my_utils.get_descriptions(conn, project_name, bug_id1)
    text2 = my_utils.get_descriptions(conn, project_name, bug_id2)
    label = "yes" if union_find.are_dups(bug_id1, bug_id2) else "No"
    dataset += [(bug_id1, text1, bug_id2, text2, label)]
    

100%|██████████| 862/862 [00:00<00:00, 3897.20it/s]


In [8]:
len(dataset)

862

In [10]:
train, test_valid = train_test_split(dataset, train_size=0.8, random_state=42)
test, valid = train_test_split(test_valid, test_size=0.5, random_state=42)

In [14]:
len(train)

689

In [20]:
folder_path = "./datasets/dup_br_data"

if not os.path.exists(folder_path):
    # If the folder doesn't exist, create it
    os.makedirs(folder_path)
    print(f'The folder "{folder_path}" has been created.')
else:
    print(f'The folder "{folder_path}" already exists.')

file_path = os.path.join(folder_path, f'{project_name}_train.csv')
with open(file_path,'w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['bug_id1','bug1','bug_id2','bug2','label'])
    for row in train:
        csv_out.writerow(row)
        
file_path = os.path.join(folder_path, f'{project_name}_test.csv')
with open(file_path,'w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['bug_id1','bug1','bug_id2','bug2','label'])
    for row in test:
        csv_out.writerow(row)
        
                
file_path = os.path.join(folder_path, f'{project_name}_test.csv')
with open(file_path,'w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['bug_id1','bug1','bug_id2','bug2','label'])
    for row in test:
        csv_out.writerow(row)

The folder "./datasets/dup_br_data" has been created.
